In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # to disable GPU
from ts_data.ts_data import ts_data as ts
import getStocks
import visualize
import ts_models.predicts
import pandas as pd
import ts_models.ts_lstm as ts_lstm

#pd.options.display.max_columns = 999
#pd.options.display.max_rows = 999

In [ ]:
# view tenslow enabled devices
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

# User parameters

In [ ]:
ticker = 'amd'.upper()
n_in = 100 # number of historical days to input
n_out = 20 # days into future to predict
target = 'd1close'

# Load Data
### Several methods available here:
* Load a single stock
* get a refresh of the stock
* load a single stock as the target while using many other stocks as additional features

## Pick the Sector and/or Industry

Purpose is to come up with a list of tickers and their corresponding exchange

In [ ]:
#d=getStocks.get_tickers_industry()
#d.to_csv('tickers_w_industry.csv')

In [ ]:
import imp
imp.reload(getStocks)

In [ ]:
# load dataset
# dataset = getStocks.get_single(ticker=ticker, save=True)
# dataset = getStocks.load_single(ticker)
dataset = getStocks.join_tgt_spt(target_ticker=ticker.upper(), industry=['Semiconductors'], exclude=['MU','QRVO'])
# dataset.reset_index(level=0, inplace=True, drop=True)
dataset.rename(columns=lambda x: x.replace(' ',''),inplace=True)

In [ ]:
dataset.head()

# Prepare Data for LSTM

In [ ]:
ts_data = ts(n_in=n_in, 
             n_out=n_out, 
             ticker=ticker,
             target=target,
             rawData=dataset)

In [ ]:
ts_data.eng_features(derivate=True, weekdays=False)

In [ ]:
ts_data.roll_data()

In [ ]:
ts_data.tscv(train=0.98)

In [ ]:
ts_data.data.tail()

# Build LSTM Architecture

In [ ]:
ts_model = ts_lstm.lstm_model(ts_data, 
                               inlayer=int(ts_data.train_X.shape[-1])*2,
                               hiddenlayers=0,
                               loss_function='mean_squared_error',
                               dropout=0.05,
                               activation='tanh',
                               gpus=1)

# Fit model

In [ ]:
import time
start = time.time()
history = ts_model.fit(ts_data.train_X, ts_data.train_y, 
                    epochs=50, 
                    batch_size=1024, 
                    validation_data=(ts_data.test_X, ts_data.test_y), 
                    verbose=2, 
                    shuffle=False)
fitTime = time.time()-start
print('Fit Time: {}'.format(round(fitTime,2)))

In [ ]:
visualize.plot_loss(history)

# Results

In [ ]:
# select the last sequence of data (it is length of n_in)
test_X = ts_data.test_X[-1]
test_Y = ts_data.test_y[-1]

In [ ]:
test_X.reshape(1, 100, 70).shape

In [ ]:
yhat = ts_model.predict(test_X.reshape(1, test_X.shape[-2], test_X.shape[-1]))

In [ ]:
import matplotlib.pyplot as plt
x = [x for x in range(len(yhat[0]))]
plt.plot(x, yhat[0], label='predicted change in price')
plt.plot(x, test_Y, label='actual change in price')
plt.title(ticker)
plt.xlabel('days')
plt.ylabel('amount change in price')
plt.legend()
plt.show()